In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

**-----------Load The Data------------**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math
import random
import numpy as np
from numpy import ndarray

In [4]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 6 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [5]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/hepmass_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [6]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, MQ)
]

In [15]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


**--------QuickSelect Algorithm----------------**

In [8]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

**----------KD Tree---------------------**

In [9]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous data storage
        if n == 2*self.leaf_size:
            self.leaf_size = n
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

**-------KD Tree Node Size Optimization For Rebuild-------**

**--------------Run 1---------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 2_500  # Ensure inclusive range
leaf_size_step = 50

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 218.01it/s]


-------------------------------------------------------------
Build Time: 15.3107 sec, Search Time: 0.7509 sec
Leaf Size: 50, Total Time: 16.0616 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 217.53it/s]


-------------------------------------------------------------
Build Time: 14.5824 sec, Search Time: 0.7296 sec
Leaf Size: 100, Total Time: 15.3120 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 11.5736 sec, Search Time: 0.7624 sec
Leaf Size: 2,400, Total Time: 12.3360 sec

49: === Testing with Leaf Size: 2,450, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 216.75it/s]


-------------------------------------------------------------
Build Time: 11.5355 sec, Search Time: 0.7751 sec
Leaf Size: 2,450, Total Time: 12.3106 sec

50: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 216.08it/s]

-------------------------------------------------------------
Build Time: 11.2813 sec, Search Time: 0.7482 sec
Leaf Size: 2,500, Total Time: 12.0295 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 50, Build Time: 15.3107 sec, Search Time: 0.7509 sec, Total Time: 16.0616 sec
Leaf Size: 100, Build Time: 14.5824 sec, Search Time: 0.7296 sec, Total Time: 15.3120 sec
Leaf Size: 150, Build Time: 14.1584 sec, Search Time: 0.7226 sec, Total Time: 14.8811 sec
Leaf Size: 200, Build Time: 13.6014 sec, Search Time: 0.7108 sec, Total Time: 14.3122 sec
Leaf Size: 250, Build Time: 13.6699 sec, Search Time: 0.7326 sec, Total Time: 14.4024 sec
Leaf Size: 300, Build Time: 13.6802 sec, Search Time: 0.7101 sec, Total Time: 14.3903 sec
Leaf Size: 350, Build Time: 13.2219 sec, Search Time: 0.7131 sec, Total Time: 13.9350 sec
Leaf Size: 400, Build Time: 12.9437 sec, Search Time: 0.6986 sec, Total Time: 13.6422 sec
Leaf Size: 450, Build Time: 12.8666 sec, Search Time: 0.7158 sec, Total Time: 13.5824

**-------------Run 2-----------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 2500  # Ensure inclusive range
leaf_size_step = 50

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 210.80it/s]


-------------------------------------------------------------
Build Time: 15.8384 sec, Search Time: 0.7610 sec
Leaf Size: 50, Total Time: 16.5995 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 222.92it/s]


-------------------------------------------------------------
Build Time: 14.9965 sec, Search Time: 0.7228 sec
Leaf Size: 100, Total Time: 15.7193 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 11.8638 sec, Search Time: 0.7678 sec
Leaf Size: 2,400, Total Time: 12.6316 sec

49: === Testing with Leaf Size: 2,450, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 210.75it/s]


-------------------------------------------------------------
Build Time: 11.6958 sec, Search Time: 0.7659 sec
Leaf Size: 2,450, Total Time: 12.4617 sec

50: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 211.49it/s]

-------------------------------------------------------------
Build Time: 11.5891 sec, Search Time: 0.7598 sec
Leaf Size: 2,500, Total Time: 12.3488 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 50, Build Time: 15.8384 sec, Search Time: 0.7610 sec, Total Time: 16.5995 sec
Leaf Size: 100, Build Time: 14.9965 sec, Search Time: 0.7228 sec, Total Time: 15.7193 sec
Leaf Size: 150, Build Time: 14.5833 sec, Search Time: 0.7360 sec, Total Time: 15.3192 sec
Leaf Size: 200, Build Time: 14.0473 sec, Search Time: 0.7172 sec, Total Time: 14.7644 sec
Leaf Size: 250, Build Time: 13.6500 sec, Search Time: 0.7251 sec, Total Time: 14.3751 sec
Leaf Size: 300, Build Time: 13.8911 sec, Search Time: 0.7210 sec, Total Time: 14.6120 sec
Leaf Size: 350, Build Time: 13.3693 sec, Search Time: 0.7147 sec, Total Time: 14.0840 sec
Leaf Size: 400, Build Time: 12.9243 sec, Search Time: 0.7086 sec, Total Time: 13.6329 sec
Leaf Size: 450, Build Time: 13.2884 sec, Search Time: 0.7205 sec, Total Time: 14.0089

**-------------Opt Node Size For Rebuild-------------**

In [ ]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array([  50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700,
  750, 800, 850, 900, 950,1000,1050,1100,1150,1200,1250,1300,1350,1400,
 1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,2050,2100,
 2150,2200,2250,2300,2350,2400,2450,2500])

# Total times from two runs (or versions)
total_times_1 = np.array([16.0616,15.312 ,14.8811,14.3122,14.4024,14.3903,13.935 ,13.6422,13.5824,
 13.7201,13.7177,13.714 ,13.6585,13.7495,13.3431,13.2167,13.3638,13.3376,
 13.3948,13.4984,13.3187,13.3099,13.0972,13.0577,13.0107,12.9142,12.7327,
 12.3993,12.7342,12.4326,12.6405,12.2784,12.258 ,12.251 ,12.2598,12.2185,
 12.3215,12.1069,12.409 ,12.0005,12.5464,12.3661,12.473 ,12.2613,12.2294,
 12.0535,12.3176,12.336 ,12.3106,12.0295])

total_times_2 = np.array([16.5995,15.7193,15.3192,14.7644,14.3751,14.612 ,14.084 ,13.6329,14.0089,
 14.0183,13.9932,13.7392,13.8079,13.7815,13.745 ,13.1   ,13.4994,13.5049,
 13.4924,13.4966,13.4201,13.2058,13.2109,13.1843,13.1131,13.0656,12.9751,
 12.758 ,13.0395,12.7247,12.5083,12.6459,12.3368,12.6089,12.6249,12.7583,
 12.6425,12.7761,12.5121,12.3599,12.6829,12.5758,12.5588,12.6068,12.3712,
 12.5776,12.6495,12.6316,12.4617,12.3488])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 12.1802
Minimum found at leaf size: 2000


**-----Epsilon Optimization For Rebuild------**

**-----------Run 1-------------------**

In [10]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size = 2000
# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.08  # Ensure inclusive range
epsilon_size_step = 0.001

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 216.68it/s]


-------------------------------------------------------------
Build Time: 10.5881 sec, Search Time: 0.7489 sec
Epsilon: 0.0, Total Time: 11.3370 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.001): 100%|██████████| 100/100 [00:00<00:00, 273.51it/s]


-------------------------------------------------------------
Build Time: 10.7894 sec, Search Time: 0.6547 sec
Epsilon: 0.001, Total Time: 11.4441 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 8.6745 sec, Search Time: 0.9715 sec
Epsilon: 0.078, Total Time: 9.6460 sec

80: === Testing with Epsilon: 0.079, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.079): 100%|██████████| 100/100 [00:00<00:00, 137.60it/s]


-------------------------------------------------------------
Build Time: 8.5604 sec, Search Time: 0.9707 sec
Epsilon: 0.079, Total Time: 9.5311 sec

81: === Testing with Epsilon: 0.08, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.08): 100%|██████████| 100/100 [00:00<00:00, 140.51it/s]

-------------------------------------------------------------
Build Time: 8.6311 sec, Search Time: 0.9444 sec
Epsilon: 0.08, Total Time: 9.5755 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 10.5881 sec, Search Time: 0.7489 sec, Total Time: 11.3370 sec
Epsilon: 0.001, Build Time: 10.7894 sec, Search Time: 0.6547 sec, Total Time: 11.4441 sec
Epsilon: 0.002, Build Time: 10.3971 sec, Search Time: 0.6311 sec, Total Time: 11.0281 sec
Epsilon: 0.003, Build Time: 10.7532 sec, Search Time: 0.6438 sec, Total Time: 11.3970 sec
Epsilon: 0.004, Build Time: 11.2656 sec, Search Time: 0.6445 sec, Total Time: 11.9100 sec
Epsilon: 0.005, Build Time: 10.6975 sec, Search Time: 0.6414 sec, Total Time: 11.3389 sec
Epsilon: 0.006, Build Time: 10.7580 sec, Search Time: 0.6594 sec, Total Time: 11.4174 sec
Epsilon: 0.007, Build Time: 10.9679 sec, Search Time: 0.6712 sec, Total Time: 11.6391 sec
Epsilon: 0.008, Build Time: 11.1212 sec, Search Time: 0.6621 sec, Total Time: 11.7833 se

**----------Run 2--------------------**

In [12]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size = 2000
# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.08  # Ensure inclusive range
epsilon_size_step = 0.001

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 216.69it/s]


-------------------------------------------------------------
Build Time: 10.9878 sec, Search Time: 0.7495 sec
Epsilon: 0.0, Total Time: 11.7373 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.001): 100%|██████████| 100/100 [00:00<00:00, 277.36it/s]


-------------------------------------------------------------
Build Time: 10.6940 sec, Search Time: 0.6478 sec
Epsilon: 0.001, Total Time: 11.3418 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 9.1439 sec, Search Time: 0.9688 sec
Epsilon: 0.078, Total Time: 10.1127 sec

80: === Testing with Epsilon: 0.079, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.079): 100%|██████████| 100/100 [00:00<00:00, 139.97it/s]


-------------------------------------------------------------
Build Time: 9.2784 sec, Search Time: 0.9746 sec
Epsilon: 0.079, Total Time: 10.2530 sec

81: === Testing with Epsilon: 0.08, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.08): 100%|██████████| 100/100 [00:00<00:00, 136.32it/s]

-------------------------------------------------------------
Build Time: 9.2251 sec, Search Time: 0.9828 sec
Epsilon: 0.08, Total Time: 10.2078 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 10.9878 sec, Search Time: 0.7495 sec, Total Time: 11.7373 sec
Epsilon: 0.001, Build Time: 10.6940 sec, Search Time: 0.6478 sec, Total Time: 11.3418 sec
Epsilon: 0.002, Build Time: 10.8598 sec, Search Time: 0.6523 sec, Total Time: 11.5120 sec
Epsilon: 0.003, Build Time: 10.8132 sec, Search Time: 0.6522 sec, Total Time: 11.4654 sec
Epsilon: 0.004, Build Time: 10.9022 sec, Search Time: 0.6614 sec, Total Time: 11.5636 sec
Epsilon: 0.005, Build Time: 11.1030 sec, Search Time: 0.6646 sec, Total Time: 11.7676 sec
Epsilon: 0.006, Build Time: 11.1696 sec, Search Time: 0.6774 sec, Total Time: 11.8471 sec
Epsilon: 0.007, Build Time: 10.7349 sec, Search Time: 0.6761 sec, Total Time: 11.4110 sec
Epsilon: 0.008, Build Time: 10.7347 sec, Search Time: 0.6786 sec, Total Time: 11.4133 s

**--------Epsilon Opt Value For Rebuild------------**

In [13]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
    [0.   ,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01 ,0.011,
 0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.02 ,0.021,0.022,0.023,
 0.024,0.025,0.026,0.027,0.028,0.029,0.03 ,0.031,0.032,0.033,0.034,0.035,
 0.036,0.037,0.038,0.039,0.04 ,0.041,0.042,0.043,0.044,0.045,0.046,0.047,
 0.048,0.049,0.05 ,0.051,0.052,0.053,0.054,0.055,0.056,0.057,0.058,0.059,
 0.06 ,0.061,0.062,0.063,0.064,0.065,0.066,0.067,0.068,0.069,0.07 ,0.071,
 0.072,0.073,0.074,0.075,0.076,0.077,0.078,0.079,0.08 ]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [11.337 ,11.4441,11.0281,11.397 ,11.91  ,11.3389,11.4174,11.6391,11.7833,
 11.2115,11.0094,11.0656,11.0057,11.0693,11.0005,11.1774,11.0762,10.8055,
 10.8269,11.5453,11.1914,10.6454,10.8523,11.0326,11.0503,11.1767,10.9783,
 10.8281,10.9221,10.475 ,10.5979,10.6288,10.4537,10.6707,10.9435,10.5936,
 10.546 ,10.5943,10.8266,10.4248,10.4964,10.6275,10.5196,10.5154,10.5286,
 10.5731,10.3379,10.2642,10.2163,10.175 , 9.9423,10.8641,10.2037,10.0553,
 10.168 ,10.6272,10.207 ,10.2706,10.1539,10.1702,10.0973, 9.8458, 9.8975,
  9.9898, 9.5592, 9.7735, 9.9377, 9.7889, 9.5311,10.0049,10.5226, 9.9045,
  9.968 , 9.6858, 9.8991, 9.7051, 9.7598, 9.8529, 9.646 , 9.5311, 9.5755]
    )

total_times_2 = np.array(
     [11.7373,11.3418,11.512 ,11.4654,11.5636,11.7676,11.8471,11.411 ,11.4133,
 11.2239,11.3634,11.2277,11.3358,11.8389,11.3501,11.4837,11.4433,11.6484,
 11.2525,11.1181,11.1255,11.126 ,10.8288,11.6828,11.022 ,11.0183,10.858 ,
 10.9687,11.0938,11.1589,11.2321,11.2363,10.9987,10.7636,10.9838,10.8809,
 11.2685,11.0336,10.9439,10.7346,11.146 ,11.1889,10.7342,10.6194,10.665 ,
 10.5836,10.6985,10.5266,10.2523,10.7064,10.9642,10.7221,10.4286,10.4673,
 10.1428,10.2662,10.0655,10.5426,10.2877, 9.938 ,10.1538,10.1143,10.291 ,
 10.5254,10.2156, 9.7482, 9.922 , 9.8803, 9.9505, 9.7796, 9.7777,10.2627,
 10.4968, 9.9883,10.0938,10.037 , 9.7367,10.1653,10.1127,10.253 ,10.2078]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 9.7408
Minimum found at epsilon size: 0.068


**------------Epsilon Optimization for Insertion-------------**

**-------------------Run 1---------------------------**

In [14]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.07  # Ensure inclusive range
epsilon_size_step = 0.001

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Readjust for the insertion
    data_size, query_size = test_combinations[0]
    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    kdtree.epsilon = epsilon

    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 149.20it/s]


-------------------------------------------------------------
Build Time: 9.0240 sec, Search Time: 1.3697 sec
Epsilon: 0.0, Total Time: 10.3937 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.001): 100%|██████████| 100/100 [00:00<00:00, 151.38it/s]


-------------------------------------------------------------
Build Time: 8.9250 sec, Search Time: 1.3074 sec
Epsilon: 0.001, Total Time: 10.2324 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 8.4873 sec, Search Time: 1.3550 sec
Epsilon: 0.068, Total Time: 9.8423 sec

70: === Testing with Epsilon: 0.069, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.069): 100%|██████████| 100/100 [00:00<00:00, 122.07it/s]


-------------------------------------------------------------
Build Time: 8.4427 sec, Search Time: 1.5009 sec
Epsilon: 0.069, Total Time: 9.9437 sec

71: === Testing with Epsilon: 0.07, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.07): 100%|██████████| 100/100 [00:00<00:00, 123.31it/s]

-------------------------------------------------------------
Build Time: 8.5240 sec, Search Time: 1.4941 sec
Epsilon: 0.07, Total Time: 10.0180 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 9.0240 sec, Search Time: 1.3697 sec, Total Time: 10.3937 sec
Epsilon: 0.001, Build Time: 8.9250 sec, Search Time: 1.3074 sec, Total Time: 10.2324 sec
Epsilon: 0.002, Build Time: 9.1954 sec, Search Time: 1.2969 sec, Total Time: 10.4923 sec
Epsilon: 0.003, Build Time: 8.9695 sec, Search Time: 1.4689 sec, Total Time: 10.4385 sec
Epsilon: 0.004, Build Time: 8.8290 sec, Search Time: 1.3570 sec, Total Time: 10.1861 sec
Epsilon: 0.005, Build Time: 8.5961 sec, Search Time: 1.3966 sec, Total Time: 9.9927 sec
Epsilon: 0.006, Build Time: 9.0140 sec, Search Time: 1.4061 sec, Total Time: 10.4201 sec
Epsilon: 0.007, Build Time: 9.0465 sec, Search Time: 1.3745 sec, Total Time: 10.4211 sec
Epsilon: 0.008, Build Time: 9.6863 sec, Search Time: 1.3440 sec, Total Time: 11.0303 sec
Epsilon

**----------------Run 2-------------------------**

In [16]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.07  # Ensure inclusive range
epsilon_size_step = 0.001

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Readjust for the insertion
    data_size, query_size = test_combinations[0]
    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    kdtree.epsilon = epsilon

    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 145.12it/s]


-------------------------------------------------------------
Build Time: 8.8603 sec, Search Time: 1.3986 sec
Epsilon: 0.0, Total Time: 10.2589 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.001): 100%|██████████| 100/100 [00:00<00:00, 146.08it/s]


-------------------------------------------------------------
Build Time: 8.9246 sec, Search Time: 1.4052 sec
Epsilon: 0.001, Total Time: 10.3298 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 8.4780 sec, Search Time: 1.4466 sec
Epsilon: 0.068, Total Time: 9.9247 sec

70: === Testing with Epsilon: 0.069, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.069): 100%|██████████| 100/100 [00:00<00:00, 120.37it/s]


-------------------------------------------------------------
Build Time: 8.5232 sec, Search Time: 1.4465 sec
Epsilon: 0.069, Total Time: 9.9697 sec

71: === Testing with Epsilon: 0.07, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.07): 100%|██████████| 100/100 [00:00<00:00, 118.96it/s]

-------------------------------------------------------------
Build Time: 8.4786 sec, Search Time: 1.5227 sec
Epsilon: 0.07, Total Time: 10.0013 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.8603 sec, Search Time: 1.3986 sec, Total Time: 10.2589 sec
Epsilon: 0.001, Build Time: 8.9246 sec, Search Time: 1.4052 sec, Total Time: 10.3298 sec
Epsilon: 0.002, Build Time: 8.9591 sec, Search Time: 1.4235 sec, Total Time: 10.3826 sec
Epsilon: 0.003, Build Time: 8.8766 sec, Search Time: 1.4671 sec, Total Time: 10.3437 sec
Epsilon: 0.004, Build Time: 9.0216 sec, Search Time: 1.4666 sec, Total Time: 10.4882 sec
Epsilon: 0.005, Build Time: 9.0392 sec, Search Time: 1.3266 sec, Total Time: 10.3658 sec
Epsilon: 0.006, Build Time: 9.2923 sec, Search Time: 1.5067 sec, Total Time: 10.7990 sec
Epsilon: 0.007, Build Time: 8.8046 sec, Search Time: 1.4632 sec, Total Time: 10.2678 sec
Epsilon: 0.008, Build Time: 9.3911 sec, Search Time: 1.4022 sec, Total Time: 10.7933 sec
Epsilo

**------Epsilon Optimization For Insertion----**

In [17]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
    [0.   ,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01 ,0.011,
 0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.02 ,0.021,0.022,0.023,
 0.024,0.025,0.026,0.027,0.028,0.029,0.03 ,0.031,0.032,0.033,0.034,0.035,
 0.036,0.037,0.038,0.039,0.04 ,0.041,0.042,0.043,0.044,0.045,0.046,0.047,
 0.048,0.049,0.05 ,0.051,0.052,0.053,0.054,0.055,0.056,0.057,0.058,0.059,
 0.06 ,0.061,0.062,0.063,0.064,0.065,0.066,0.067,0.068,0.069,0.07 ]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [10.2589,10.3298,10.3826,10.3437,10.4882,10.3658,10.799 ,10.2678,10.7933,
 10.3356,10.2682,10.2931,10.1916,10.3505,10.6959,10.3199,10.3622,10.367 ,
 10.4347,10.4435,10.7239, 9.9559,10.322 ,10.3571,10.281 ,10.5912,10.3184,
 10.3015,10.5986,10.591 ,10.2281,10.5539,10.5754,11.3666,10.3979,10.2485,
 10.5571,10.6379,10.3313,10.5666,10.4069,10.2435,10.3613,10.1729,10.1424,
 10.1807,10.647 ,10.4044,10.215 ,10.0246,10.4721,10.0909,10.2561,10.1714,
  9.9965,10.204 ,10.1941, 9.7448, 9.9856,10.0364,10.1295,10.2621,10.4383,
 10.2491,10.0882,10.2112, 9.9329,10.2392, 9.9247, 9.9697,10.0013]
    )

total_times_2 = np.array(
    [10.3937,10.2324,10.4923,10.4385,10.1861, 9.9927,10.4201,10.4211,11.0303,
 10.2571,10.7068,10.5084,10.232 ,10.5634,10.4436,10.2679,10.3073,10.393 ,
 10.4694,10.5761,10.0602,10.2965,10.3196,10.2504,10.5555,10.6221,10.0836,
 10.1631,10.4439,10.1478,10.0836,10.2725,10.0911,10.5662,10.0279,10.1642,
 10.5813,10.3759,10.3259,10.4409,10.0602,10.1779,10.1976,10.1837,10.3668,
  9.9068,10.0699,10.1193,10.1691,10.0247,10.0073,10.1577,10.0983,10.1189,
 10.0498, 9.9828,10.0942,10.2038,10.1205,10.2253, 9.8974,10.1168, 9.9801,
 10.206 ,10.1829,10.103 ,10.1202,10.1055, 9.8423, 9.9437,10.018 ]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 9.8835
Minimum found at epsilon size: 0.068
